In [1]:
!pip install -q kagglehub numpy pandas networkx matplotlib torch torch-geometric==2.4.0 torch-geometric-temporal


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [137]:
!pip install -q torch-geometric==2.6.1


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
%cd NYSE-Temporal-Graph-Construction

/teamspace/studios/this_studio/NYSE-Temporal-Graph-Construction


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [138]:
import networkx as nx
from matplotlib import pyplot as plt
from tqdm import tqdm
from os.path import join as join_path
import os
from torch.utils.data import Dataset
import kagglehub
import pandas as pd
from torch_geometric_temporal import DynamicGraphTemporalSignal, Data
from functools import cache
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cpu


In [4]:
TARGET_GRAPH_DIR = "graph_data/train/infotech_thresh1/"
FILTER_BY_WEIGHT_THRESH = 5

## Node features

In [5]:
dataset_path = kagglehub.dataset_download("dgawlik/nyse")

In [87]:
fundamentals_df = pd.read_csv(join_path(dataset_path,"fundamentals.csv"))
prices_split_adjusted_df = pd.read_csv(join_path(dataset_path, "prices-split-adjusted.csv"))
prices_df = pd.read_csv(join_path(dataset_path, "prices.csv"))
securities_df = pd.read_csv(join_path(dataset_path, "securities.csv"))

In [162]:
all_symbols = prices_split_adjusted_df['symbol'].unique()

@cache
def symbol_to_int(symbol: str):
    return all_symbols.tolist().index(symbol)

@cache
def int_to_symbol(idx):
    return all_symbols[idx]

print("Num of symbols:", len(all_symbols))

Num of symbols: 501


In [89]:
prices_split_adjusted_df['date'] = pd.to_datetime(prices_split_adjusted_df['date']).dt.date
dates = prices_split_adjusted_df['date'].sort_values().unique()



def any_to_date(date):
    if not isinstance(date, pd._libs.tslibs.timestamps.Timestamp):
        date = pd.to_datetime(date).date()
    return date

@cache
def date_to_int(date):
    date = any_to_date(date)
    return dates.tolist().index(date)

@cache
def int_to_date(idx):
    return dates[idx]

print(dates)

[datetime.date(2010, 1, 4) datetime.date(2010, 1, 5)
 datetime.date(2010, 1, 6) ... datetime.date(2016, 12, 28)
 datetime.date(2016, 12, 29) datetime.date(2016, 12, 30)]


In [120]:
node_features_df = prices_split_adjusted_df.copy()
node_features_df['day_diff'] = ((node_features_df['close']) - (o:=node_features_df['open'])) / o
node_features_df['date'] = node_features_df['date'].apply(lambda x: date_to_int(x))
node_features_df['symbol'] = node_features_df['symbol'].apply(lambda x: symbol_to_int(x))

# rename symbol to node_id
node_features_df = node_features_df.rename(columns={'symbol': 'node', 'date': 'time'})



node_features_df.sort_values(['time', 'node'], inplace=True)
node_features_df.head(10)


,time,node,open,close,low,high,volume,day_diff
251,0,1,22.453504,22.389128,22.267525,22.625180,3815500.0,-0.002867
252,0,2,4.840000,4.770000,4.660000,4.940000,9837300.0,-0.014463
253,0,3,40.700001,40.380001,40.360001,41.040001,1701700.0,-0.007862
254,0,4,30.490000,30.572857,30.340000,30.642857,123432400.0,0.002718
255,0,5,26.290001,26.629999,26.139999,26.690001,2455900.0,0.012933
256,0,6,26.000339,26.129884,25.870792,26.177866,10829000.0,0.004982
257,0,7,41.520000,42.070000,41.500000,42.200001,3650100.0,0.013247
258,0,8,36.650002,37.090000,36.650002,37.299999,4710200.0,0.012005
259,0,9,31.790001,31.670000,31.610001,32.189999,2102700.0,-0.003775
260,0,10,31.480000,31.469999,31.330000,31.840000,3472500.0,-0.000318


## Graph

In [56]:
graphs = [nx.read_weighted_edgelist(join_path(TARGET_GRAPH_DIR, fileName)) for fileName in os.listdir(TARGET_GRAPH_DIR)]

In [103]:
print("Total num of graphs:", len(graphs))
print("Total num of edges:", sum([graph.number_of_edges() for graph in graphs]))

Total num of graphs: 1233
Total num of edges: 2438163


In [164]:
torch_graphs = []
for time, nx_graph in enumerate(graphs):
    today_node_features_df = node_features_df[node_features_df['time'] == time].set_index('node', inplace=False)
    tomorrow_node_features_df = node_features_df[node_features_df['time'] == time + 1].set_index('node', inplace=False)

    today_node_features_df = today_node_features_df.reindex(range(len(all_symbols)), fill_value=0.0)
    tomorrow_node_features_df = tomorrow_node_features_df.reindex(range(len(all_symbols)), fill_value=0.0)

    today_node_features_df = today_node_features_df.drop(columns=['time'])

    y = tomorrow_node_features_df['day_diff'].values
    y = torch.tensor(y, device=device, dtype=torch.float)

    # convert to torch
    edges_data = pd.DataFrame.from_dict(nx_graph.edges.data())

    edges_data[0] = edges_data[0].apply(lambda x: symbol_to_int(x))
    edges_data[1] = edges_data[1].apply(lambda x: symbol_to_int(x))
    edges_data[2] = edges_data[2].apply(lambda x: x['weight'])

    # Filter by weight
    edges_data = edges_data[edges_data[2] > FILTER_BY_WEIGHT_THRESH]
    
    edge_index = edges_data.loc[:,0:1].values
    edge_index = torch.tensor(edge_index, dtype=torch.long, device=device).t().contiguous()

    edge_weight = edges_data.loc[:,2].values
    edge_weight = torch.tensor(edge_weight, dtype=torch.float, device=device)

    torch_graph = Data(x=today_node_features_df, edge_index=edge_index, edge_attr=edge_weight, y=y,)
    torch_graphs.append(torch_graph)

## Dataset

In [165]:
print("Graphs len:", len(torch_graphs))
# print("Node features len:", len(torch_node_features))

Graphs len: 1233


In [141]:
# dataset = DynamicGraphTemporalSignal(
#     [tg.edge_index for tg in torch_graphs],
#     [tg.edge_attr for tg in torch_graphs],
#     [tg.x for tg in torch_graphs],
#     [tg.y for tg in torch_graphs],
# )

In [166]:
from torch.utils.data import Dataset

class NYSEDataset(Dataset):
    def __init__(self, torch_graphs):
        self.graphs = torch_graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]


dataset = NYSEDataset(torch_graphs)

In [179]:
for time, snapshot in enumerate(dataset):
    print(snapshot)

    if time == 10:
        break

Data(edge_index=[2, 740], edge_attr=[740], y=[501])
Data(edge_index=[2, 645], edge_attr=[645], y=[501])
Data(edge_index=[2, 738], edge_attr=[738], y=[501])
Data(edge_index=[2, 997], edge_attr=[997], y=[501])
Data(edge_index=[2, 819], edge_attr=[819], y=[501])
Data(edge_index=[2, 925], edge_attr=[925], y=[501])
Data(edge_index=[2, 573], edge_attr=[573], y=[501])
Data(edge_index=[2, 696], edge_attr=[696], y=[501])
Data(edge_index=[2, 802], edge_attr=[802], y=[501])
Data(edge_index=[2, 634], edge_attr=[634], y=[501])
Data(edge_index=[2, 775], edge_attr=[775], y=[501])


In [27]:
print("Total num of edges after weight filter:", sum([graph.number_of_edges() for graph in graphs]))

Total num of edges after weight filter: 907564


In [ ]:
import torch
import torch.nn.functional as F
import torch_geometric.
from torch_geometric_temporal.nn.recurrent import GCLSTM

class RecurrentGCN(torch.nn.Module):

    def __init__(self, node_features, num_classes):
        super(RecurrentGCN, self).__init__()
        self.recurrent_1 = GCLSTM(node_features, 32, 5)
        self.recurrent_2 = GCLSTM(32, 16, 5)
        self.linear = torch.nn.Linear(16, num_classes)

    def forward(self, x, edge_index, edge_weight):
        x = self.recurrent_1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.recurrent_2(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear(x)
        return F.log_softmax(x, dim=1)